In [100]:
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
import pandas as pd
import helper_functions as hf
from constants import KAGGLE_DIR
%reload_ext autoreload
%autoreload 2

In [101]:
%aimport helper_functions

## Add some team spellings to MTeamSpellings.csv manually

This is needed for joining our scraped data (sports-reference) with our provided data (kaggle). Think the best/quickest way to do this is just manually right now.

"Spellings" I'm manually adding to the MTeamSpellings.csv (which didn't have matches with the sports-reference dataset):
- california-baptist (1465)
- dixie-state (1469)
- north-alabama (1466)
- st-thomas-mn (1472)
- tarleton-state (1460)
- california-san-diego (1471)

In [102]:
unmatched_spellings_lst = [('california-baptist', 1465), ('dixie-state', 1469), ('north-alabama', 1466),
                           ('st-thomas-mn', 1472), ('tarleton-state', 1460), ('california-san-diego', 1471)]
hf.update_team_spellings_file(unmatched_spellings_lst)

In [103]:
team_spellings = pd.read_csv(f'{KAGGLE_DIR}/MTeamSpellings.csv', encoding='unicode_escape')

In [104]:
unmatched_spellings_lst = [('california-baptist', 1465), ('dixie-state', 1469), ('north-alabama', 1466),
                           ('st-thomas-mn', 1472), ('tarleton-state', 1460), ('california-san-diego', 1471)]
unmatched_spellings = pd.DataFrame({
    'TeamNameSpelling': [spelling for spelling, _ in unmatched_spellings_lst],
    'TeamID': [teamid for _, teamid in unmatched_spellings_lst]
})
unmatched_spellings

,TeamNameSpelling,TeamID
0,california-baptist,1465
1,dixie-state,1469
2,north-alabama,1466
3,st-thomas-mn,1472
4,tarleton-state,1460
5,california-san-diego,1471


In [105]:
full_spellings = pd.concat([team_spellings, unmatched_spellings], ignore_index=True)
full_spellings.drop_duplicates(inplace=True)
full_spellings.to_csv(f'{KAGGLE_DIR}/MTeamSpellings.csv', index=False)

In [107]:
team_spellings = pd.read_csv(f'{KAGGLE_DIR}/MTeamSpellings.csv', encoding='unicode_escape')
team_spellings

,TeamNameSpelling,TeamID
0,a&m-corpus chris,1394
1,a&m-corpus christi,1394
2,abilene chr,1101
3,abilene christian,1101
4,abilene-christian,1101
...,...,...
1166,north-alabama,1466
1167,st-thomas-mn,1472
1168,tarleton-state,1460
1169,california-san-diego,1471


In [62]:
print(yr)

2021


In [57]:
link = 'https://www.sports-reference.com/cbb/seasons/' + str(yr) + '-coaches.html'
with urllib.request.urlopen(link) as url:
    page = url.read()
soup = BeautifulSoup(page, "html.parser")
coach_table = soup.find('table', {'id': 'coaches'})
coach_tbody = coach_table.find('tbody')
coach_rows = coach_tbody.find_all('tr')
cols = ['Season','school','is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25','coach_WL_car','tourneys_car','sw16_car','ff_car','champ_car']
coach_df = pd.DataFrame(columns=cols)

In [108]:
for coach_row in coach_rows:
    tds = coach_row.find_all('td')
    if len(tds) > 10:
        try:
            school = tds[0].find('a')['href'].split('/')[3]
            is_ap_pre_top_5 = False if tds[6].text == '' else int(tds[6].text) <= 5
            is_ap_pre_top_15 = False if tds[6].text == '' else 5 < int(tds[6].text) <= 15
            is_ap_pre_top_25 = False if tds[6].text == '' else 15 < int(tds[6].text) <= 25
            # arbitrarily choosing 0.300 winning % if the coach for some reason has no winning % listed (probably a bad sign)
            coach_WL_car = 0.3 if tds[21].text == '' else float(tds[21].text)
            tourneys_car = 0 if tds[22].text == '' else int(tds[22].text)
            sw16_car = 0 if tds[23].text == '' else int(tds[23].text)
            ff_car = 0 if tds[24].text == '' else int(tds[24].text)
            champ_car = 0 if tds[25].text == '' else int(tds[25].text)
            new_row = pd.DataFrame(columns=cols)
            new_row.loc[0] = [yr, school, is_ap_pre_top_5, is_ap_pre_top_15, is_ap_pre_top_25, coach_WL_car, tourneys_car, sw16_car, ff_car, champ_car]
            coach_df = pd.concat([coach_df, new_row], ignore_index=True)
        except:
            print("Skip bad row")

Skip bad row


In [86]:
yr

2021

## Scrape coaching data from sports-reference

In [109]:
def get_coach_df(yr):
    link = 'https://www.sports-reference.com/cbb/seasons/' + str(yr) + '-coaches.html'
    with urllib.request.urlopen(link) as url:
        page = url.read()
    soup = BeautifulSoup(page, "html.parser")
    coach_table = soup.find('table', {'id': 'coaches'})
    coach_tbody = coach_table.find('tbody')
    coach_rows = coach_tbody.find_all('tr')
    cols = ['Season','school','is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25','coach_WL_car','tourneys_car','sw16_car','ff_car','champ_car']
    coach_df = pd.DataFrame(columns=cols)
    for coach_row in coach_rows:
        tds = coach_row.find_all('td')
        if len(tds) > 10:
            try:
                school = tds[0].find('a')['href'].split('/')[3]
                is_ap_pre_top_5 = False if tds[6].text == '' else int(tds[6].text) <= 5
                is_ap_pre_top_15 = False if tds[6].text == '' else 5 < int(tds[6].text) <= 15
                is_ap_pre_top_25 = False if tds[6].text == '' else 15 < int(tds[6].text) <= 25
                # arbitrarily choosing 0.300 winning % if the coach for some reason has no winning % listed (probably a bad sign)
                coach_WL_car = 0.3 if tds[21].text == '' else float(tds[21].text)
                tourneys_car = 0 if tds[22].text == '' else int(tds[22].text)
                sw16_car = 0 if tds[23].text == '' else int(tds[23].text)
                ff_car = 0 if tds[24].text == '' else int(tds[24].text)
                champ_car = 0 if tds[25].text == '' else int(tds[25].text)
                new_row = pd.DataFrame(columns=cols)
                new_row.loc[0] = [yr, school, is_ap_pre_top_5, is_ap_pre_top_15, is_ap_pre_top_25, coach_WL_car, tourneys_car, sw16_car, ff_car, champ_car]
                coach_df = pd.concat([coach_df, new_row], ignore_index=True)
            except:
                print("Skip bad row")
    return(coach_df)

In [110]:
def sports_ref_join_to_kaggle(coach_df):
    team_spellings = pd.read_csv(f'{KAGGLE_DIR}/MTeamSpellings.csv')
    coach_joined = team_spellings.merge(coach_df, left_on='TeamNameSpelling', right_on='school')
    coach_joined.drop('TeamNameSpelling', axis=1, inplace=True)
    return(coach_joined)

In [111]:
def check_for_missing_spellings(coach_df, coach_joined):
    coach_comp = coach_df.merge(coach_joined, on='school', how='left')
    return(coach_comp[coach_comp['TeamNameSpelling'].isna()])

## Compiling dataset

In [112]:
coach_df = get_coach_df(2021)
coach_df

Skip bad row


,Season,school,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car
0,2021,kennesaw-state,False,False,False,0.113,0,0,0,0
1,2021,lipscomb,False,False,False,0.525,0,0,0,0
2,2021,longwood,False,False,False,0.442,0,0,0,0
3,2021,belmont,False,False,False,0.591,1,0,0,0
4,2021,nevada,False,False,False,0.651,11,4,0,0
...,...,...,...,...,...,...,...,...,...,...
350,2021,southern,False,False,False,0.442,1,0,0,0
351,2021,villanova,True,False,False,0.691,17,7,3,2
352,2021,illinois-chicago,False,False,False,0.409,0,0,0,0
353,2021,fairfield,False,False,False,0.373,0,0,0,0


In [114]:
coach_df = get_coach_df(2003)
coach_joined = sports_ref_join_to_kaggle(coach_df)
for yr in range(2004,2024):
    print(yr)
    new_coach_df = get_coach_df(yr)
    new_coach_joined = sports_ref_join_to_kaggle(new_coach_df)
    coach_joined = pd.concat([coach_joined, new_coach_joined], ignore_index=True)

HTTPError: HTTP Error 429: Too Many Requests

In [55]:
coach_joined.head()

,TeamID,Season,school,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,0


In [21]:
coach_joined[coach_joined['Season'] == 2022]

,TeamID,Season,school,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car
6601,1101,2022,abilene-christian,False,False,False,0.697,0,0,0,0
6602,1102,2022,air-force,False,False,False,0.474,1,0,0,0
6603,1103,2022,akron,False,False,False,0.592,4,1,0,0
6604,1104,2022,alabama,False,True,False,0.668,5,1,0,0
6605,1105,2022,alabama-am,False,False,False,0.290,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6954,1460,2022,wright-state,False,False,False,0.587,5,0,0,0
6955,1461,2022,wyoming,False,False,False,0.633,1,0,0,0
6956,1462,2022,xavier,False,False,False,0.580,0,0,0,0
6957,1463,2022,yale,False,False,False,0.547,3,0,0,0


In [22]:
coach_joined.to_csv('data/generated_data/coach_features.csv', index=False)

## Some data checks

In [27]:
coach_joined[coach_joined['Season'] == 2022].iloc[140:170]

,TeamID,Season,school,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car
6741,1240,2022,jacksonville-state,False,False,False,0.599,4,0,0,0
6742,1241,2022,james-madison,False,False,False,0.574,0,0,0,0
6743,1242,2022,kansas,True,False,False,0.768,23,13,3,1
6744,1243,2022,kansas-state,False,False,False,0.622,13,4,1,0
6745,1244,2022,kennesaw-state,False,False,False,0.226,0,0,0,0
6746,1245,2022,kent-state,False,False,False,0.612,1,0,0,0
6747,1246,2022,kentucky,False,True,False,0.771,21,15,6,1
6748,1247,2022,la-salle,False,False,False,0.388,0,0,0,0
6749,1248,2022,lafayette,False,False,False,0.455,3,0,0,0
6750,1249,2022,lamar,False,False,False,0.335,0,0,0,0
